In [2]:
import os


import pprint
import random
import numpy as np
import pandas as pd

import spacy
import sys
sys.path.append("../")
from datatools.analyzer import *
from datatools.maneger import DataManager
# from datatools.analyzer import clean_text
from error_tools import *

In [3]:
from nltk.lm        import Vocabulary
# from nltk.lm.models import MLE
from nltk.lm.models import KneserNeyInterpolated
from nltk.util      import ngrams

In [164]:
from datatools.maneger import DataManager
n=3
modelM = DataManager("../models/utterance/")
model_name = "KLM_phrase_wiki_n={0}_splited.pickle".format(n)
model_name = "KLM_phrase_nucc_ntt_n={0}_splited.pickle".format(n)
model_name = "KLM_phrase_nucc_ntt_wiki_n={0}_splited.pickle".format(n)
model_name = "KLM_phrase_nucc_ntt_wiki_n={0}_splited_unidic.pickle".format(n)
lm = modelM.load_data(model_name)

success load : ../models/utterance/KLM_phrase_nucc_ntt_wiki_n=3_splited_unidic.pickle


In [105]:
path = "../eval_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Grammatical error"
errors = ['Grammatical error', "Uninterpretable"]
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            if not ut.utt[-1] in ["？", "！", "。", "!"]:
                sys_utt.append( clean_text( ut.utt+"。" ))
                # sys_utt.append(ut.utt)
            else:   
                sys_utt.append(clean_text( ut.utt ))
            if ut.is_error_included(errors):
                y.append(1)
            else:
                y.append(0)

In [174]:
mecab_analyzer = Analyzer(tokenizer_)
mecab_analyzer = Analyzer(Tokenizer())

def last_pos(pos_split):
    last = len(pos_split) - 1 
    for i, _ in enumerate(pos_split):
        if pos_split[last-i] != "*":
            return "["+pos_split[last-i]+"]"
# def sentence2normalize_noun_mecab(sentences):
#     normalize_sen = []
#     if isinstance(sentences, str):
#         sentences = [sentences]
#     for sen in sentences:
#         df = mecab_analyzer.analyze_with_dataframe(sen)
#         words = []
#         for txt, pos in zip(df.surface, df.part_of_speech):
#             pos_split = pos.split("-")
#             # if pos_split[0]
#             # if pos_split[1] != "サ変接続" and pos_split[1] != "非自立" and pos_split[1] != "接尾" and "名詞" == pos_split[0]:
#             if pos_split[1] == "固有名詞" or pos_split[1] == "一般":
#                 # words.append(pos)
#                 words.append(last_pos(pos_split))
#             else:
#                 words.append(txt)
#         normalize_sen.append(words)
#     return normalize_sen

def sentence2normalize_noun_mecab(sentences):
    normalize_sen = []
    if isinstance(sentences, str):
        sentences = [sentences]
    for sen in sentences:
        df = mecab_analyzer.analyze_with_dataframe(sen)
        words = []
        if df is None:
            continue
        for txt, pos in zip(df.surface, df.part_of_speech):
            pos_split = pos.split("-")
            # if pos_split[0]
            # if pos_split[1] != "サ変接続" and pos_split[1] != "非自立" and pos_split[1] != "接尾" and "名詞" == pos_split[0]:
            print(pos_split)
            if pos_split[0]=="名詞" and ( pos_split[1] == "固有名詞" or pos_split[1] == "一般"):
                # words.append(pos)
                # print(txt, pos)
                words.append(last_pos(pos_split))
            else:
                words.append(txt)
        normalize_sen.append(words)
    return normalize_sen


mecab_analyzer = Analyzer(Tokenizer())

def last_pos(pos_split, filler="*"):
    last = len(pos_split) - 1 
    for i, _ in enumerate(pos_split):
        if pos_split[last-i] != filler :
            return "["+pos_split[last-i]+"]"
def sentence2normalize_noun_mecab(sentences):
    normalize_sen = []
    if isinstance(sentences, str):
        sentences = [sentences]
    for sen in sentences:
        df = mecab_analyzer.analyze_with_dataframe(sen)
        words = []
        if df is None:
            continue
        for txt, pos in zip(df.surface, df.part_of_speech):
            pos_split = pos.split("-")
            # if pos_split[0]
            # if pos_split[1] != "サ変接続" and pos_split[1] != "非自立" and pos_split[1] != "接尾" and "名詞" == pos_split[0]:
            if pos_split[0]=="名詞":
                # words.append(pos)
                # print(txt, pos)
                last_pos_name = last_pos(pos_split, filler="")
                if last_pos_name=="[サ変可能]":
                    words.append(txt)
                else:
                    words.append(last_pos_name)
            else:
                words.append(txt)
        normalize_sen.append(words)
    return normalize_sen

In [166]:
import math
import pprint


def sentence2score(sentence, lm, N, is_print=False):
    # filled = fill_SYMBOL( sentence2normalize_independent(sentence) )
    # filled = fill_SYMBOL_ONE( sentence2normalize_noun_mecab(sentence) )
    filled = sentence2normalize_noun_mecab(sentence)
    normal_split = [ list("".join(L)) for L in filled ]
    # filled = fill_SYMBOL( sentence2morpheme(sentence) )
    # filled_pos = fill_SYMBOL_ONE( sentence2pos_mecab(sentence) )
    # print(filled)
    # print(filled_pos)
    ngram_text = []
    ngram_pos = []
    # for L, P in zip(filled):
    for L in normal_split:
        for i in range(len(L)-N+1):
            # print(L[i:i+N])
            ngram_text.append(L[i:i+N])
            # ngram_pos.append(P[i:i+N])
    # pprint.pprint(ngram_text)
    all_score = 0
    function_score = 0
    # デフォルトで1
    function_num = 1

    under = 1 / (1000*len( lm.vocab.counts ) )

    min_score = 0

    # for ngram, pgram in zip(ngram_text, ngram_pos):
    for ngram in ngram_text:
        context = (ngram[:-1])
        # context_pos = pgram[:-1]
        # print(context)
        # for word in lm.context_counts(lm.vocab.lookup(context)): # 文脈に続く単語一覧の取得
            
        score = lm.score(ngram[-1], context) + under
        log_score = math.log2(score)
        if is_print:
            print("context : {0}|{1} ->".format(context, ngram[-1:]), log_score)
        if log_score < min_score:
            min_score = log_score

        # if "助動詞" in context_pos[1] or "助詞" in context_pos[1] or "助動詞" in context_pos[0] or "助詞" in context_pos[0]:
        #     # print("\tcontext : {0}| ->".format(context), log_score)
        #     function_score += log_score
        #     function_num += 1

        all_score += log_score
    # print(all_score/len(ngram_text))
    # return all_score/len(ngram_text)
    return  min_score

In [171]:
sentence = "熱中症に気をつけか??"
# sentence = "最近とても暑いですから。"
# sentence = "仲間由紀恵は怖い"
sentence2score(clean_text(sentence), lm, N=n, is_print=True)

100%|██████████| 1/1 [00:00<00:00, 480.89it/s]

context : ['熱', '中']|['症'] -> -1.8407896861385864
context : ['中', '症']|['に'] -> -0.3576363069212646
context : ['症', 'に']|['['] -> -2.0228209876331347
context : ['に', '[']|['一'] -> -0.4191514837509937
context : ['[', '一']|['般'] -> 3.433652350548337e-07
context : ['一', '般']|[']'] -> 3.433652350548337e-07
context : ['般', ']']|['を'] -> -3.7795003803728964
context : [']', 'を']|['つ'] -> -6.351459545586087
context : ['を', 'つ']|['け'] -> -0.5926746717796643
context : ['つ', 'け']|['か'] -> -8.711270979131637
context : ['け', 'か']|['?'] -> -3.628992130214225
context : ['か', '?']|['?'] -> -7.329583962284409


-8.711270979131637

In [172]:
sentence2normalize_noun_mecab(sentence)

100%|██████████| 1/1 [00:00<00:00, 293.72it/s]


[['熱中', '症', 'に', '[一般]', 'を', 'つけ', 'か', '?', '?']]

In [175]:
y_scores = []
for utt in tqdm(sys_utt):
    y_scores.append(sentence2score(utt, lm, N=n))

100%|██████████| 1386/1386 [00:02<00:00, 472.16it/s]


In [176]:
def f1_score_plain(test, pred):
    return f1_score(y_true=test, y_pred=pred)

def recall_score_plain(test, pred):
    return recall_score(y_true=test, y_pred=pred)

def precision_score_plain(test, pred):
    return precision_score(y_true=test, y_pred=pred)

In [177]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score
import numpy as np

base = -5
epoch = 1000
# y_pred = np.zeros(len(y)) + 1
y_pred = np.zeros(len(y))
max_f1 = 0

print(model_name)
for e in tqdm(range(epoch)):
    y_pred = np.zeros(len(y))

    for i, score_ in enumerate(y_scores):
        # border 未満をエラーでとする
        border = (base - 0.1*e)
        if score_ < border :
            y_pred[i] = 1
    f1 = f1_score_plain(y, y_pred)
    if f1 > max_f1:
        max_f1 = f1
        print("f1 score:{0}, border:{1}".format(max_f1, border))

  6%|▌         | 61/1000 [00:00<00:01, 607.26it/s]

KLM_phrase_nucc_ntt_wiki_n=3_splited_unidic.pickle
f1 score:0.010471204188481674, border:-5.0
f1 score:0.010510510510510511, border:-5.1
f1 score:0.01051840721262209, border:-5.2
f1 score:0.010534236267870579, border:-5.3
f1 score:0.01061410159211524, border:-5.4
f1 score:0.010630220197418374, border:-5.5
f1 score:0.010654490106544902, border:-5.6
f1 score:0.010687022900763359, border:-5.7
f1 score:0.010760953112990006, border:-5.8
f1 score:0.01078582434514638, border:-5.9
f1 score:0.010819165378670788, border:-6.0
f1 score:0.010920436817472701, border:-6.1
f1 score:0.010989010989010988, border:-6.2
f1 score:0.011014948859166013, border:-6.3
f1 score:0.011182108626198084, border:-6.4
f1 score:0.011217948717948718, border:-6.5
f1 score:0.011235955056179775, border:-6.6
f1 score:0.011336032388663968, border:-6.7
f1 score:0.01138211382113821, border:-6.8
f1 score:0.011494252873563216, border:-6.9
f1 score:0.011527377521613832, border:-12.100000000000001
f1 score:0.011661807580174927, bord

 18%|█▊        | 178/1000 [00:00<00:01, 543.29it/s]

f1 score:0.01663201663201663, border:-15.8
f1 score:0.01702127659574468, border:-15.9
f1 score:0.017582417582417582, border:-16.0
f1 score:0.017937219730941704, border:-16.1
f1 score:0.018475750577367205, border:-16.200000000000003
f1 score:0.01900237529691211, border:-16.3
f1 score:0.01904761904761905, border:-17.200000000000003
f1 score:0.019801980198019802, border:-17.3
f1 score:0.020477815699658702, border:-17.4
f1 score:0.021428571428571432, border:-17.5
f1 score:0.022641509433962263, border:-17.6
f1 score:0.022988505747126436, border:-18.5
f1 score:0.024999999999999998, border:-18.6
f1 score:0.026666666666666665, border:-18.700000000000003
f1 score:0.028169014084507043, border:-18.8
f1 score:0.03571428571428571, border:-19.8
f1 score:0.0392156862745098, border:-20.0
f1 score:0.04, border:-20.1
f1 score:0.043478260869565216, border:-20.200000000000003
f1 score:0.05714285714285715, border:-20.3
f1 score:0.06451612903225806, border:-20.4
f1 score:0.06666666666666667, border:-20.5
f1

100%|██████████| 1000/1000 [00:01<00:00, 663.26it/s]


In [190]:
y_pred2 = np.zeros(len(y))

for i, score_ in enumerate(y_scores):
    if score_ < -17 :
        y_pred2[i] = 1

In [191]:
score(y, y_pred2)

confusion matrix = 
 [[1052  327]
 [   4    3]]
accuracy =  0.7611832611832612
precision =  0.00909090909090909
recall =  0.42857142857142855
f1 score =  0.01780415430267062


In [187]:
for p, y_ , utt in zip(y_pred2, y, sys_utt):
    if y_==1 and p==0:
        print(utt)

熱中症に気をつけか??
熱中症に気をつけか??
withha。
